## Сбор датасета

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from time import sleep
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import re
import string
from sklearn.feature_extraction.text  import TfidfVectorizer
from sklearn.metrics  import f1_score, accuracy_score
from sklearn.metrics import  confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import ADASYN
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.linear_model import SGDClassifier
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.base import clone
from sklearn.model_selection import cross_validate
from scipy import interp
from sklearn.metrics import roc_curve, auc 
import warnings
warnings.filterwarnings("ignore")

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
def parse_product(product, bank='tcs', page_limit=100):
  '''
  Downloads (RSHB bank) reviews  about needed product
  return DataFrame with columns:
  time, product, title, full_text, mark
  '''
  base_url = f'https://www.banki.ru/services/responses/bank/{bank}/product/{product}/'
  r = requests.get(base_url)
  soup = BeautifulSoup(r.text)
  pages = soup.find('div', {'data-test': 'folkrating-responses-list-comments-pagination'}).get('data-options')
  cur_page, items_per_page, all_items = [int(s.rstrip(';')) for s in pages.split() if s.rstrip(';').isdigit()]
  last_page_items = all_items % items_per_page
  num_pages = all_items // items_per_page + bool(last_page_items)
  data_df = []
  for page in range(1, num_pages+1):
    if page > page_limit:
      continue
    sleep(2)
    url = f'{base_url}?page={page}'
    r_temp = None
    while r_temp is None:
      try:
        r_temp = requests.get(url)
      except OSError:
        sleep(10)
    r = r_temp
    print(f'page {page}/{num_pages} scrapped succesfully')
    soup = BeautifulSoup(r.text)
    articles = soup.find_all('article', {'class': 'responses__item'})
    if page != num_pages:
      n_articles = items_per_page
    else:
      n_articles = last_page_items
    for article in range(n_articles):
      time = articles[article].find('time', {'class': 'display-inline-block'}).text.strip()
      title = articles[article].div.text
      full_text = articles[article].find('div', \
                              {'class': 'responses__item__message markup-inside-small markup-inside-small--bullet'})
      full_text = full_text.text.strip()
      mark = articles[article].find('div', 
      {'class': 'flexbox flexbox--inline flexbox--row flexbox--gap_xsmall flexbox--align-items_baseline'}).text.strip()
      new_row = {'time':time, 'product':product, 'title':title, 'full_text':full_text, 'mark':mark}
      data_df.append(new_row)

  df = pd.DataFrame(data_df)
  return df


Таблица соответствий названий классов (видов услуг) и части ссылки product: 
*   Дебетовая карта - **debitcards**
*   Ипотека - **hypothec**
*   Дистанционное обслуживание физ. лиц - **remote**
*   Дистанционное обслуживание юр. лиц - **business_remote**
*   Обслуживание юр. лиц - **corporate**


Код ниже загружает необходимые виды услуг и сохраняет в csv, из-за ограничений сайта Банки.ру загрузка занимает более 10 минут

In [ ]:
debitcards = parse_product(product='debitcards')
hypothec = parse_product(product='hypothec')
remote = parse_product(product='remote')
business_remote = parse_product(product='business_remote')
corporate = parse_product(product='corporate')



debitcards.to_csv('debitcards.csv', sep='\t')
hypothec.to_csv('hypothec.csv', sep='\t')
remote.to_csv('remote.csv', sep='\t')
business_remote.to_csv('business_remote.csv', sep='\t')
corporate.to_csv('corporate.csv', sep='\t')

In [ ]:
all = [debitcards, hypothec, remote, business_remote, corporate]
alldf = pd.concat(all)
alldf

,time,product,title,full_text,mark
0,15.06.2021 9:28,debitcards,\nГорячая линия (не мог войти в приложение)\n,Сегодня впервые воспользовался горячей линией ...,Оценка:\n\n\t\t\t5\n\t\t\n\n\t\t\t\t\t\t\tПров...
1,15.06.2021 9:23,debitcards,\nСАМЫЙ НЕБЕЗОПАСНЫЙ БАНК!!\n,Тинькофф заморозили счета до 5 июля .Служба бе...,Оценка:\n\n\t\t\t1\n\t\t\n\n\t\t\t\t\t\t\tПров...
2,15.06.2021 8:58,debitcards,\nБыл вход с другого устройства. Списали деньг...,Пользовался другим банком до этого таких пробл...,Оценка:\n\n\t\t\t1\n\t\t\n\n\t\t\t\t\t\t\tПров...
3,15.06.2021 8:57,debitcards,\nБанк дорожит своими клиентами\n,"Я пенсионер, возраст 60+. Пользуюсь только бе...",Оценка:\n\n\t\t\t5\n\t\t\n\n\t\t\t\t\t\t\tПров...
4,15.06.2021 8:50,debitcards,"\nПрекрасный банк, который не стыдно посоветов...","Знакомство с банком у меня началось с того, чт...",Оценка:\n\n\t\t\t5\n\t\t\n\n\t\t\t\t\t\t\tПров...
...,...,...,...,...,...
2042,26.05.2015 23:53,corporate,\nНеуважительное отношение\n,"Много слышал историй про Банк Тинькофф, хороши...",Без оценки
2043,20.05.2015 19:24,corporate,\nТы мог был быть хорошим другом...\n,Предисловие.Были чудные времена когда банком и...,Оценка:\n\n\t\t\t3\n\t\t\n\n\nПроблема решена
2044,19.05.2015 13:21,corporate,\nРекламный спам\n,Банк занимается рекламным спамом. Звонят и пре...,Без оценки
2045,20.08.2014 16:54,corporate,"\nМучения с банком ТКС, ужасная оперативность....","Мне лично нравится Тиньков, я смотрел передачи...",Без оценки


In [ ]:
alldf.shape

(9764, 5)

In [ ]:
alldf.to_csv('alldf.csv', sep='\t')